In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import json
from PIL import Image

In [4]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.img_names = [img for img in os.listdir(img_dir) if img.endswith('.jpg')]

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_name = self.img_names[idx]
        img_path = os.path.join(self.img_dir, img_name)
        
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"No such file: '{img_path}'")
        
        image = Image.open(img_path)

        parts = img_name.split('_')
        pants_type = f"{parts[0]}_{parts[1]}_{parts[2]}"  # 예: "long_pants_denim"
        file_num = int(parts[4])  # 'long_pants_denim_image_124_1.jpg' 형태라고 가정

        label = self.dataframe[(self.dataframe['file_num'] == file_num) & 
                               (self.dataframe['pants_type'] == pants_type)].iloc[0, 1]

        if self.transform:
            image = self.transform(image)
        
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label


In [6]:
# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Lambda(lambda x: x.convert('RGB')),  # RGB로 변환
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
# 데이터프레임 로드 및 준비 과정
directory_path = 'D:/minkwan/무신사 크롤링/coordikitty-ML-DL/압축/롱숏/라벨'
data = []

for file_name in os.listdir(directory_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'r') as file:
            content = json.load(file)
            pants_type = file_name.split('_label_')[1].split('.json')[0]  # 파일명에서 pants_type 추출
            # 여기에서 pants_type을 문자열로 처리합니다.
            # 예: "long_pants_denim"
            data.append({
                "file_num": content["file_num"],
                "medium_category": content["medium_category"],
                "pants_type": pants_type  # pants_type 컬럼을 문자열로 추가
            })

df = pd.DataFrame(data)
df = df.sort_values(by=['file_num'])
df = df.reset_index(drop=True)
df

,file_num,medium_category,pants_type
0,1,롱팬츠,long_pants_denim
1,1,롱팬츠,long_pants_sweat
2,1,숏팬츠,short_pants_denim
3,1,숏팬츠,short_pants_sweat
4,2,롱팬츠,long_pants_denim
...,...,...,...
4651,1378,롱팬츠,long_pants_denim
4652,1379,롱팬츠,long_pants_denim
4653,1380,롱팬츠,long_pants_denim
4654,1381,롱팬츠,long_pants_denim


In [8]:
#레이블 인코딩
df['medium_category'] = pd.factorize(df['medium_category'])[0]

img_dir = 'D:/minkwan/무신사 크롤링/coordikitty-ML-DL/압축/롱숏/이미지'

dataset = CustomDataset(dataframe=df, img_dir=img_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
df

,file_num,medium_category,pants_type
0,1,0,long_pants_denim
1,1,0,long_pants_sweat
2,1,1,short_pants_denim
3,1,1,short_pants_sweat
4,2,0,long_pants_denim
...,...,...,...
4651,1378,0,long_pants_denim
4652,1379,0,long_pants_denim
4653,1380,0,long_pants_denim
4654,1381,0,long_pants_denim


In [9]:
# 모델 정의 및 조정
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, df['medium_category'].nunique())
model = model.to(device)

c:\Users\mkmy7\anaconda3\envs\Pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mkmy7\anaconda3\envs\Pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# 손실 함수 및 옵티마이저 설정
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:
# 모델 이어서 학습을 위한 체크포인트 파일 경로
checkpoint_path = 'model_resnet50 copy.pth'

# 이어서 학습할 경우 체크포인트 로드
start_epoch = 0
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1

In [102]:
# 학습
print_every = 100  # 출력 빈도 조정
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)


        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % print_every == (print_every - 1):
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / print_every}')
            running_loss = 0.0
print('Finished Training')

[1, 100] loss: 0.5292556710634381
[1, 200] loss: 0.3838448735496786
[1, 300] loss: 0.13494434064283267
[1, 400] loss: 0.20226913013088052
[1, 500] loss: 0.08938525381774526
[1, 600] loss: 0.1358770762972563
[2, 100] loss: 0.06299236457663937
[2, 200] loss: 0.08107456133475352
[2, 300] loss: 0.15475411897379673
[2, 400] loss: 0.12085589416295989
[2, 500] loss: 0.020884912051187712
[2, 600] loss: 0.04126969160453882
[3, 100] loss: 0.02610390051453578
[3, 200] loss: 0.033035482199775285
[3, 300] loss: 0.03298644834976585
[3, 400] loss: 0.010022826975837234
[3, 500] loss: 0.011162834249917068
[3, 600] loss: 0.010328124414008925
[4, 100] loss: 0.01575687265569286
[4, 200] loss: 0.0053625048769754356
[4, 300] loss: 0.011910569356869018
[4, 400] loss: 0.004925762680545631
[4, 500] loss: 0.0023840082224660364
[4, 600] loss: 0.013520982760169317
[5, 100] loss: 0.003535481553562931
[5, 200] loss: 0.0015643280834592588
[5, 300] loss: 0.0015545896166804596
[5, 400] loss: 0.0015227446891526598
[5, 

In [103]:
# 모델 저장
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, 'model_resnet50 copy.pth')

In [13]:
#테스트 코드
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 정의 및 조정
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # 예시로 10개의 클래스가 있다고 가정
model = model.to(device)

# 모델 로드
checkpoint_path = 'D:/minkwan/무신사 크롤링/coordikitty-ML-DL/model_resnet50_중분류.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 라벨 인덱스를 medium_category로 매핑
class_names = df['medium_category'].unique()
class_names = sorted(class_names, key=lambda x: list(df['medium_category']).index(x))
idx_to_class = {i: class_name for i, class_name in enumerate(class_names)}

def predict_image_category(image_path, model, transform, device, idx_to_class):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_idx = predicted.item()
        category = idx_to_class[predicted_idx]
    
    return category

# 테스트할 이미지 경로
test_image_path = 'D:/minkwan/무신사 크롤링/coordikitty-ML-DL/압축/롱팬츠_코튼/long_pants_cotton_test_data_90개/image_8_2.jpg'

# 예측 수행
predicted_category = predict_image_category(test_image_path, model, transform, device, idx_to_class)
if predicted_category == 0:
    print(f'The predicted medium_category is: 롱팬츠')
elif predicted_category == 1:
    print(f'The predicted medium_category is: 숏팬츠')

The predicted medium_category is: 롱팬츠
